# Mlflow setup test

This notebook is used to test the mlflow setup with a dummy classification experiment.

In [1]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("file:../mlruns")  # Adjust path as needed if notebook is nested
mlflow.set_experiment("Test_Classification1")


<Experiment: artifact_location='file:///Users/bobhosseini/Portfolio/SDS-social-sphere/submissions/team-members/bob-hosseini/notebooks/../mlruns/809692970443597525', creation_time=1749839751561, experiment_id='809692970443597525', last_update_time=1749839751561, lifecycle_stage='active', name='Test_Classification1', tags={}>

In [2]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import mlflow.sklearn

# Minimal dummy data
X = pd.DataFrame({"feature1": [1, 0, 1, 0], "feature2": [0, 1, 1, 0]})
y = [1, 0, 1, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

with mlflow.start_run():
    clf = DummyClassifier(strategy="most_frequent")
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    acc = accuracy_score(y_test, preds)
    
    mlflow.log_param("strategy", "most_frequent")
    mlflow.log_metric("accuracy", acc)
    mlflow.sklearn.log_model(clf, "dummy_model", input_example=X_test.iloc[0:1])


2025/06/14 19:38:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/anaconda3/envs/c29env/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
